In [ ]:
import boto3
import psycopg2

# Set up credentials
aws_access_key_id = '<AWS_ACCESS_KEY_ID>'
aws_secret_access_key = '<AWS_SECRET_ACCESS_KEY>'
region_name = '<REGION_NAME>'
cluster_identifier = '<CLUSTER_IDENTIFIER>'
database_name = '<DATABASE_NAME>'
table_name = '<TABLE_NAME>'
csv_file_path = '<CSV_FILE_PATH>'
iam_role = '<IAM_ROLE_ARN>'

# Create Redshift client
redshift = boto3.client('redshift', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Create Redshift cluster
redshift.create_cluster(
    ClusterIdentifier=cluster_identifier,
    NodeType='dc2.large',
    MasterUsername='admin',
    MasterUserPassword='Password123',
    ClusterSubnetGroupName='default',
    VpcSecurityGroupIds=['<SECURITY_GROUP_ID>'],
    PubliclyAccessible=True,
)

# Wait for cluster to become available
redshift.get_waiter('cluster_available').wait(ClusterIdentifier=cluster_identifier)

# Create database table
with psycopg2.connect(
    host=redshift.describe_clusters(ClusterIdentifier=cluster_identifier)['Clusters'][0]['Endpoint']['Address'],
    port=5439,
    dbname=database_name,
    user='admin',
    password='Password123',
) as conn:
    with conn.cursor() as cur:
        cur.execute(f"CREATE TABLE {table_name} (col1 VARCHAR(50), col2 VARCHAR(50), col3 INT)")
        conn.commit()

# Load data from CSV file into table
with psycopg2.connect(
    host=redshift.describe_clusters(ClusterIdentifier=cluster_identifier)['Clusters'][0]['Endpoint']['Address'],
    port=5439,
    dbname=database_name,
    user='admin',
    password='Password123',
) as conn:
    with conn.cursor() as cur:
        cur.execute(f"COPY {table_name} FROM '{csv_file_path}' IAM_ROLE '{iam_role}' DELIMITER ',' CSV HEADER")
        conn.commit()